## Word Embedding


In this notebook, I will present and practice a simple, probabilistic method by [Mikolov et al., 2013] : [**Word2Vec**](https://web.stanford.edu/class/cs224n/readings/cs224n-2019-notes01-wordvecs1.pdf). Word2Vec and other word embedding tools is to solve OneHot to represent word as well as the relationships. It represent every word as a fixed length vector and through traning, these vectors can have semantic meanings. In order to compute the language model, we need to calculate the probability of words and the conditional probability of a word given other few words.

Word2vec is a software package that actually includes :

- 2 algorithms: **[Continuous Bag-Of-Words (CBOW)](https://web.stanford.edu/class/cs224n/readings/cs224n-2019-notes01-wordvecs1.pdf)** and **[Skip-Gram](https://web.stanford.edu/class/cs224n/readings/cs224n-2019-notes01-wordvecs1.pdf)**

**CBOW** aims to predict a center word from the surrounding context in terms of word vectors. It assumes the center $w_c$ is generated by context $P(w_c\mid \mathcal{W}_o)$, where $\mathcal{W}_o$ is the set of context words.

![Image Name](https://cdn.kesci.com/upload/image/q5mjt4r02n.png?imageView2/0/w/960/h/960)



**Skip-Gram** does the opposite, assume the surrounding words $w_o$ are generated by center $w_c$, that is $P(w_o\mid w_c)$ and predicts the distribution (probability) of context words from a center word.

![Image Name](https://cdn.kesci.com/upload/image/q5mjsq84o9.png?imageView2/0/w/960/h/960)


- 2 training methods: **[Negative Sampling](https://web.stanford.edu/class/cs224n/readings/cs224n-2019-notes01-wordvecs1.pdf)** and **[Hierarchical Softmax](https://web.stanford.edu/class/cs224n/readings/cs224n-2019-notes01-wordvecs1.pdf)**

**Negative Sampling** defines an objective by sampling negative examples, while **H-softmax** defines an objective using an efficient. In practice, hierarchical softmax tends to be better for infrequent words, while negative sampling works better for frequent words and lower dimensional vectors.

#### This notebook will focus on Skip-Gram implementation and Negative Sampling to train Word2Vec model using PTB dataset. 

In [1]:
import collections
import math
import random
import sys
import time
import os
import numpy as np
import torch
from torch import nn
import torch.utils.data as Data

### PTB dataset

Simply put，Word2Vec can learn how to map discrete words to vectors in continuous space from a large corpus while keeping its semantic meaning. 

So to train our Word2Vec model, in this notebook I will use a NLP corpus called PTB [PTB (Penn Tree Bank)](https://catalog.ldc.upenn.edu/LDC99T42). PTB is a commonly-used small size corpus, coming from articles on Wall Street Journal, including training set, test set and validation set. I will train the word embedding on PTB train set.

- load dataset : take train file `ptb.train.txt` as an example：
```
aer banknote berlitz calloway centrust cluett fromstein gitano guterman ...
pierre  N years old will join the board as a nonexecutive director nov. N 
mr.  is chairman of  n.v. the dutch publishing group 
...
```

In [2]:
with open('/home/kesci/input/ptb_train1020/ptb.train.txt', 'r') as f:
    lines = f.readlines() # separated by line
    raw_dataset = [st.split() for st in lines] # split the words in sentence by space

print('# sentences: %d' % len(raw_dataset))

# sentences: 42068
# tokens: 24 ['aer', 'banknote', 'berlitz', 'calloway', 'centrust']
# tokens: 15 ['pierre', '<unk>', 'N', 'years', 'old']
# tokens: 11 ['mr.', '<unk>', 'is', 'chairman', 'of']


### Preprocessing

This part illustrates the process how to split text data into tokens, and then map these discrete tokens to index in our vocab. 
 
 - Split sentence into tokens
 
 For each sentence, we first split it into a list of tokens. A token is a data point the model will train and predict. 
 
 
 - Create word index

 The string type of the token is inconvenient to be used by models, which take numerical inputs. So we need a dictionary, often called vocabulary as well, to map string tokens into numerical indices starting from 0. To do so, we first count the unique tokens in all documents, called corpus, and then assign a numerical index to each unique token according to its frequency. In our vocab, we filter out low-freq words to reduce the complexity. 
 
 In some cases, we also use special tokens, which are “<unk>”, “<pad>”, “<bos>” and “<eos>” 

In [ ]:
# print length and first 5 words of first 3 sents
# eos token as '' ，unk token as '' , replace number as 'N'
for st in raw_dataset[:3]:
    print('# tokens:', len(st), st[:5])

In [3]:
counter = collections.Counter([tk for st in raw_dataset for tk in st]) 
counter = dict(filter(lambda x: x[1] >= 5, counter.items())) # filter low-freq words apprear less than 5 times

idx_to_token = [tk for tk, _ in counter.items()]
token_to_idx = {tk: idx for idx, tk in enumerate(idx_to_token)} # map token word with index
dataset = [[token_to_idx[tk] for tk in st if tk in token_to_idx]
           for st in raw_dataset] # create vocab with idx of token words in raw_dataset 
num_tokens = sum([len(st) for st in dataset])
'# tokens: %d' % num_tokens

'# tokens: 887100'

### Subsampling

Subsampling is a technique to remove some high-freq words like "the", "a", "in" in text data to improve model performance. As in some cases, these words just add unnecessary noise to the model and can distort it from finding a true signal. 

The detail of subsampling in word2vec is that the random removal of tokens is done before the corpus is processed into word-context pairs. (i.e. before word2vec is actually run) In practice, it is similar to filter out stop words, by performing a resampling to get a dataset, in which all words have a certain prob dropped out in training, which is:

$$

P(w_i)=\max(1-\sqrt{\frac{t}{f(w_i)}},0)

$$


where $f(w_i)$ is the frequency of word $w_i$ in subsampling  (a.k.a ${count(𝑤𝑖)}$/ ${total_Count}$ ) and $t$ is a hyperparameter, a chosen threshold (set at $10^{−4}$ in this notebook). So, it is only possible to discard $w_i$ in subsampling only when $f(w_i) > t$, and the higher freq of $w_i$, the higher the probability of being discarded.

In [4]:
def discard(idx):
    '''
    @params:
        idx: subscript of the word token
    @return: True/False drop the token or not
    '''
    return random.uniform(0, 1) < 1 - math.sqrt(
        1e-4 / counter[idx_to_token[idx]] * num_tokens)

subsampled_dataset = [[tk for tk in st if not discard(tk)] for st in dataset]
print('# tokens: %d' % sum([len(st) for st in subsampled_dataset]))

def compare_counts(token):
    return '# %s: before=%d, after=%d' % (token, 
    sum([st.count(token_to_idx[token]) for st in dataset]), 
    sum([st.count(token_to_idx[token]) for st in subsampled_dataset]))

print(compare_counts('the'))
print(compare_counts('join'))

# tokens: 375548
# the: before=50770, after=2131
# join: before=45, after=45


We can see that after subsampling, the freq of word "the" has reduced greatly and that of "join" hasn't changed. And we didn't throw away token from vocab, instead just discard randomly in the sentences. So this function works well!

### Extract center word and contexts

In [5]:
def get_centers_and_contexts(dataset, max_window_size):
    '''
    @params:
        dataset: all token from original dataset by index
        max_window_size: max context window size
    @return:
        centers: list of center words
        contexts: list of context words matched with centers
    '''
    centers, contexts = [], []
    
    for st in dataset:
        if len(st) < 2:  # at least 2 words to create center-context
            continue
        centers += st
        
        for center_i in range(len(st)):
            window_size = random.randint(1, max_window_size) # random choose context window size
            indices = list(range(max(0, center_i - window_size),
                                 min(len(st), center_i + 1 + window_size)))
            indices.remove(center_i)  # remove center from context words
            contexts.append([st[idx] for idx in indices])
    
    return centers, contexts 

all_centers, all_contexts = get_centers_and_contexts(subsampled_dataset, 5)

tiny_dataset = [list(range(7)), list(range(7, 10))]
print('dataset', tiny_dataset)

for center, context in zip(*get_centers_and_contexts(tiny_dataset, 2)):
    print('center', center, 'has contexts', context)

dataset [[0, 1, 2, 3, 4, 5, 6], [7, 8, 9]]
center 0 has contexts [1]
center 1 has contexts [0, 2, 3]
center 2 has contexts [1, 3]
center 3 has contexts [2, 4]
center 4 has contexts [2, 3, 5, 6]
center 5 has contexts [3, 4, 6]
center 6 has contexts [5]
center 7 has contexts [8, 9]
center 8 has contexts [7, 9]
center 9 has contexts [8]


*Note: The batch size data processing requires negtive sampling, which will be implemented later in this notebook.*

### Skip-Gram Model

In this model, each word is represented as two $d$ -dim vector to calculate condition probability. 

If the word has an index of $i$ in the vocab, when it is the center, the vector is $\boldsymbol{v}_i\in\mathbb{R}^d$，while when it is in the context, the vector is  $\boldsymbol{u}_i\in\mathbb{R}^d$ 

Let the index of the center word $w_c$ in the vocab is $c$，context $w_o$ indices are $o$ , the condition probability of generating context $w_o$ given center $w_c$ is:


$$

P(w_o\mid w_c)=\frac{\exp(\boldsymbol{u}_o^\top \boldsymbol{v}_c)}{\sum_{i\in\mathcal{V}}\exp(\boldsymbol{u}_i^\top \boldsymbol{v}_c)}


$$



- PyTorch built-in embedding layer

In [6]:
# used to get token by index
embed = nn.Embedding(num_embeddings=10, embedding_dim=4)  # define a layer with num of token and embedding dim
print(embed.weight)

x = torch.tensor([[1, 2, 3], [4, 5, 6]], dtype=torch.long)
print(embed(x))

Parameter containing:
tensor([[-1.3367,  1.2066,  0.1861, -0.7905],
        [-1.4386,  0.2864,  0.8527,  1.6524],
        [ 0.4732,  0.1352, -0.5633, -0.3785],
        [ 1.5265, -1.0352,  1.1603,  1.3615],
        [-0.7230, -1.0227, -0.8679, -0.5138],
        [ 0.6433,  0.9655,  0.2737,  0.1145],
        [-0.2912, -1.1775, -1.3287, -0.1709],
        [ 1.7273,  0.9296, -0.8355, -0.1213],
        [-1.3989, -0.0632,  1.0090, -1.7358],
        [ 0.5273, -1.2383,  0.7594, -0.8828]], requires_grad=True)
tensor([[[-1.4386,  0.2864,  0.8527,  1.6524],
         [ 0.4732,  0.1352, -0.5633, -0.3785],
         [ 1.5265, -1.0352,  1.1603,  1.3615]],

        [[-0.7230, -1.0227, -0.8679, -0.5138],
         [ 0.6433,  0.9655,  0.2737,  0.1145],
         [-0.2912, -1.1775, -1.3287, -0.1709]]], grad_fn=<EmbeddingBackward>)


- PyTorch built-in batch matrix multiplication

In [7]:
X = torch.ones((2, 1, 4))
Y = torch.ones((2, 4, 6))
print(torch.bmm(X, Y).shape)

torch.Size([2, 1, 6])


Now let's define forward function for our Skip Gram Model.

In [8]:
def skip_gram(center, contexts_and_negatives, embed_v, embed_u):
    '''
    @params:
        center: center index，shape (n, 1) int tensor
        contexts_and_negatives: indices of contexts and negatives，shape (n, m) int tensor
        embed_v: embedding layer for center
        embed_u: embedding layer for contexts
    
    @return:
        pred: batch matrix multiplication of center and contexts to calculate prob later p(w_o|w_c)
    '''
    v = embed_v(center) # shape of (n, 1, d)
    u = embed_u(contexts_and_negatives) # shape of (n, m, d)
    pred = torch.bmm(v, u.permute(0, 2, 1)) # reshape bmm (n, 1, d) to (n, d, m)
    
    return pred # shape of (n, 1, m)

### Negative Sampling

As Softmax computation for all tokens in a large NLP corpus would be too expensive, we will solve this issue using negative sampling on our skip-gram model. For every training step, instead of looping over the entire vocabulary, we can just sample several negative examples! We "sample" from a noise distribution (Pn(w)) whose probabilities match the ordering of the frequency of the vocabulary.

Negative sampling idea is based on the concept of noise contrastive estimation, which indicates that a good model should differentiate fake signal from the real one by the means of logistic regression. Also, the motivation behind negative sampling objective is similar to stochastic gradient descent: instead of changing all of the weights each time with taking into account all of the thousands of observations we’re having, we’re using only K of them and increasing computational efficiency.

Negative Sampling uses this formula to simulate condition probability: $P(w_o\mid w_c)=\frac{\exp(\boldsymbol{u}_o^\top \boldsymbol{v}_c)}{\sum_{i\in\mathcal{V}}\exp(\boldsymbol{u}_i^\top \boldsymbol{v}_c)}$：

$$P(w_o\mid w_c)=P(D=1\mid w_c,w_o)\prod_{k=1,w_k\sim P(w)}^K P(D=0\mid w_c,w_k)$$

Where $P(D=1\mid w_c,w_o)=\sigma(\boldsymbol{u}_o^\top\boldsymbol{v}_c)$，$\sigma(\cdot)$ is the sigmoid function

For a pair of center and contexts,  we randomly sample $K$ negatives from vocab (here, $K=5$), a common practice is to set the negative sampling prob $P(w)$ to the $0.75$ power of the ratio of word freq $w$ to the total word frequency.

In [9]:
def get_negatives(all_contexts, sampling_weights, K):
    '''
    @params:
        all_contexts: [[w_o1, w_o2, ...], [...], ... ]
        sampling_weights: all word's negative sampling probability
        K: num of random samples
    @return:
        all_negatives: [[w_n1, w_n2, ...], [...], ...]
    '''
    all_negatives, neg_candidates, i = [], [], 0
    population = list(range(len(sampling_weights)))
    for contexts in all_contexts:
        negatives = []
        while len(negatives) < len(contexts) * K:
            if i == len(neg_candidates):
                # generate indices of k negatives according to each word's sampling_weights
                # use larger k for computation efficiency
                i, neg_candidates = 0, random.choices(
                    population, sampling_weights, k=int(1e5))
            neg, i = neg_candidates[i], i + 1
            
            # negatives cannot be contexts
            if neg not in set(contexts):
                negatives.append(neg)
        all_negatives.append(negatives)
    return all_negatives

sampling_weights = [counter[w]**0.75 for w in idx_to_token]
all_negatives = get_negatives(all_contexts, sampling_weights, 5)

* Apart from Negative Sampling, **Hierarchical softmax** can also be used to solve the computation problem.

### Load batch data

In [10]:
class MyDataset(torch.utils.data.Dataset):
    
    def __init__(self, centers, contexts, negatives):
        assert len(centers) == len(contexts) == len(negatives)
        self.centers = centers
        self.contexts = contexts
        self.negatives = negatives
        
    def __getitem__(self, index):
        return (self.centers[index], self.contexts[index], self.negatives[index])

    def __len__(self):
        return len(self.centers)
    
def batchify(data):
    '''
    param collate_fn for DataLoader
    @params:
        data: list length of batch_size, each element is output from __getitem__
    @outputs:
        batch: output (centers, contexts_negatives, masks, labels) tuple
            centers: idx of center, shape (n, 1) int tensor
            contexts_negatives: idx of contexts and negatives，shape (n, m) int tensor
            masks: mask corresponding to padding in loss function, shape is (n, m), 0/1 int tensor
            labels: label for centers，shape is (n, m), 0/1 int tensor
    '''
    max_len = max(len(c) + len(n) for _, c, n in data)
    centers, contexts_negatives, masks, labels = [], [], [], []
    for center, context, negative in data:
        cur_len = len(context) + len(negative)
        centers += [center]
        contexts_negatives += [context + negative + [0] * (max_len - cur_len)]
        masks += [[1] * cur_len + [0] * (max_len - cur_len)] # use mask variable to cover padding in loss function
        labels += [[1] * len(context) + [0] * (max_len - len(context))]
        batch = (torch.tensor(centers).view(-1, 1), torch.tensor(contexts_negatives),
            torch.tensor(masks), torch.tensor(labels))
    return batch

batch_size = 512
num_workers = 0 if sys.platform.startswith('win32') else 4

dataset = MyDataset(all_centers, all_contexts, all_negatives)
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True,
                            collate_fn=batchify, 
                            num_workers=num_workers)
for batch in data_iter:
    for name, data in zip(['centers', 'contexts_negatives', 'masks',
                           'labels'], batch):
        print(name, 'shape:', data.shape)
    break

centers shape: torch.Size([512, 1])
contexts_negatives shape: torch.Size([512, 60])
masks shape: torch.Size([512, 60])
labels shape: torch.Size([512, 60])


## Model Training

- Loss function

After applying the negative sampling, we can use the log-equivalent form of MLE (Maximum Likelihood Estimation) to define the loss function as follows:

$$

\sum_{t=1}^T\sum_{-m\le j\le m,j\ne 0} [-\log P(D=1\mid w^{(t)},w^{(t+j)})-\sum_{k=1,w_k\sim P(w)^K}\log P(D=0\mid w^{(t)},w_k)]

$$


According to its definition, we then use the binary cross entropy loss function to calculate the loss

In [11]:
class SigmoidBinaryCrossEntropyLoss(nn.Module):
    def __init__(self):
        super(SigmoidBinaryCrossEntropyLoss, self).__init__()
    def forward(self, inputs, targets, mask=None):
        '''
        @params:
            inputs: the prob of D=1 after sigmoid function
            targets: 0/1 vector，1 is background，0 is negative
        @return:
            res: the avg loss per label
        '''
        inputs, targets, mask = inputs.float(), targets.float(), mask.float()
        res = nn.functional.binary_cross_entropy_with_logits(inputs, targets, reduction="none", weight=mask)
        res = res.sum(dim=1) / mask.float().sum(dim=1)
        return res

loss = SigmoidBinaryCrossEntropyLoss()

pred = torch.tensor([[1.5, 0.3, -1, 2], [1.1, -0.6, 2.2, 0.4]])
label = torch.tensor([[1, 0, 0, 0], [1, 1, 0, 0]]) # label 1: context; 0: negatives
mask = torch.tensor([[1, 1, 1, 1], [1, 1, 1, 0]])  # mask variable
print(loss(pred, label, mask))

def sigmd(x):
    return - math.log(1 / (1 + math.exp(-x)))
    
print('%.4f' % ((sigmd(1.5) + sigmd(-0.3) + sigmd(1) + sigmd(-2)) / 4)) # 1-sigmoid(x) = sigmoid(-x)
print('%.4f' % ((sigmd(1.1) + sigmd(-0.6) + sigmd(-2.2)) / 3))

tensor([0.8740, 1.2100])
0.8740
1.2100


In [12]:
# initialize the model
embed_size = 100
net = nn.Sequential(nn.Embedding(num_embeddings=len(idx_to_token), embedding_dim=embed_size),
                    nn.Embedding(num_embeddings=len(idx_to_token), embedding_dim=embed_size))

### Training

In [13]:
def train(net, lr, num_epochs):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print("train on", device)
    net = net.to(device)
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    
    for epoch in range(num_epochs):
        start, l_sum, n = time.time(), 0.0, 0
        for batch in data_iter:
            center, context_negative, mask, label = [d.to(device) for d in batch]
            
            pred = skip_gram(center, context_negative, net[0], net[1])
            
            l = loss(pred.view(label.shape), label, mask).mean() # avg loss for one batch
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            l_sum += l.cpu().item()
            n += 1
        
        print('epoch %d, loss %.2f, time %.2fs'
              % (epoch + 1, l_sum / n, time.time() - start))

train(net, 0.01, 5)

train on cpu
epoch 1, loss 1.98, time 672.59s
epoch 2, loss 0.62, time 670.56s
epoch 3, loss 0.45, time 670.54s
epoch 4, loss 0.40, time 671.01s
epoch 5, loss 0.37, time 669.49s


### Testing

In [14]:
def get_similar_tokens(query_token, k, embed):
    '''
    @params:
        query_token: token
        k: num of similar words
        embed: pre-trained word vector
    '''
    W = embed.weight.data
    x = W[token_to_idx[query_token]]
    cos = torch.matmul(W, x) / (torch.sum(W * W, dim=1) * torch.sum(x * x) + 1e-9).sqrt() # eps for smoothing
    _, topk = torch.topk(cos, k=k+1)
    topk = topk.cpu().numpy()
    for i in topk[1:]:  # remove input word
        print('cosine sim=%.3f: %s' % (cos[i], (idx_to_token[i])))
        
get_similar_tokens('chip', 3, net[0])


cosine sim=0.453: counseling
cosine sim=0.441: software
cosine sim=0.434: center


In [29]:
get_similar_tokens('stock', 3, net[0])

cosine sim=0.645: exchange
cosine sim=0.627: shares
cosine sim=0.587: friday


In [33]:
get_similar_tokens('girl', 3, net[0])

cosine sim=0.399: glamorous
cosine sim=0.387: advised
cosine sim=0.380: affected


In [34]:
get_similar_tokens('man', 3, net[0])

cosine sim=0.526: says
cosine sim=0.513: <unk>
cosine sim=0.470: politburo


In [40]:
get_similar_tokens('shopping', 3, net[0])

cosine sim=0.409: magazines
cosine sim=0.394: coffee
cosine sim=0.394: personnel


From above examples, we can see that the language model is not perfect and not generalized to other themes due to my CPU trainging limit and the size and content of our text corpus. However, it gave us some sort of sense of how word embedding like word2vec can achieve in NLP. In future, it is good to start from here and implement more complex word embedding on larger corpus as well as to explore the combinnation with other modern advanced language models!

#### References
* [Dive into Deep Learning](https://d2l.ai/chapter_natural-language-processing/word2vec.html)
* [CS224n: Natural Language Processing with Deep Learning 1](https://web.stanford.edu/class/cs224n/readings/cs224n-2019-notes01-wordvecs1.pdf)
* [Dive-into-DL-PyTorch on GitHub](https://github.com/ShusenTang/Dive-into-DL-PyTorch/blob/master/code/chapter10_natural-language-processing/10.3_word2vec-pytorch.ipynb)